## Assignment 1
## 2a. Logistic regression with MNIST

In [42]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import sys
sys.path.append(os.getcwd()+"/examples")


In [43]:
import numpy as np
import tensorflow as tf
import time

import utils

In [44]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 30

In [45]:
# Step 1: Read in data
mnist_folder = './examples/data/mnist'
utils.download_mnist(mnist_folder)
train, val, test = utils.read_mnist(mnist_folder, flatten=True)

Successfully downloaded ./examples/data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded ./examples/data/mnist/train-labels-idx1-ubyte.gz
Successfully downloaded ./examples/data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded ./examples/data/mnist/t10k-labels-idx1-ubyte.gz


In [46]:
# Step 2: Create datasets and iterator
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)	# initializer for train_data
test_init = iterator.make_initializer(test_data)	# initializer for train_data

In [48]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
w = tf.get_variable(name='weights_logres_mnist', shape=(784, 10), initializer=tf.random_normal_initializer(0, 0.01))
b = tf.get_variable(name='bias_logres_mnist', shape=(1, 10), initializer=tf.zeros_initializer())

In [49]:
# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
logits = tf.matmul(img, w) + b 

In [50]:
# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label, name='entropy_logres_mnist')
loss = tf.reduce_mean(entropy, name='loss_logres_mnist') # computes the mean over all the examples in the batch

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [51]:
# Step 6: define training op
# using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [52]:
# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

In [53]:
writer = tf.summary.FileWriter('./graphs/logreg', tf.get_default_graph())
with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass

    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()

Average loss epoch 0: 0.36774478066106175
Average loss epoch 1: 0.29374980458686517
Average loss epoch 2: 0.2852804884487806
Average loss epoch 3: 0.2796804225028947
Average loss epoch 4: 0.27423563512951826
Average loss epoch 5: 0.2722181519450143
Average loss epoch 6: 0.2743374989822853
Average loss epoch 7: 0.2702030371094859
Average loss epoch 8: 0.26762627453305
Average loss epoch 9: 0.2654736254450887
Average loss epoch 10: 0.26412032995113105
Average loss epoch 11: 0.25963915184486747
Average loss epoch 12: 0.2622866994766302
Average loss epoch 13: 0.261157258738612
Average loss epoch 14: 0.2600917337938797
Average loss epoch 15: 0.26032566643038463
Average loss epoch 16: 0.2597665125200915
Average loss epoch 17: 0.2592495939066244
Average loss epoch 18: 0.25675975633568543
Average loss epoch 19: 0.2594437779555487
Average loss epoch 20: 0.2542842911426411
Average loss epoch 21: 0.2585429338348466
Average loss epoch 22: 0.256637892646845
Average loss epoch 23: 0.2536856445115666